In [1]:
!pip freeze

asttokens==2.4.1
colorama==0.4.6
comm==0.2.0
debugpy==1.8.0
decorator==5.1.1
exceptiongroup==1.1.3
executing==2.0.1
ipykernel==6.26.0
ipython==8.17.2
jedi==0.19.1
joblib==1.3.2
jupyter_client==8.6.0
jupyter_core==5.5.0
matplotlib-inline==0.1.6
nest-asyncio==1.5.8
numpy==1.26.2
packaging==23.2
pandas==2.1.3
parso==0.8.3
platformdirs==4.0.0
prompt-toolkit==3.0.40
psutil==5.9.6
pure-eval==0.2.2
Pygments==2.16.1
python-dateutil==2.8.2
pytz==2023.3.post1
pywin32==306
pyzmq==25.1.1
scikit-learn==1.3.2
scipy==1.11.3
six==1.16.0
stack-data==0.6.3
threadpoolctl==3.2.0
tornado==6.3.3
traitlets==5.13.0
tzdata==2023.3
wcwidth==0.2.9


In [2]:
import pandas as pd

In [28]:
df  = pd.read_csv('../data/credit_card/fraudTrain.csv')

# Assuming 'df' is your DataFrame and 'timestamp_column' is the name of your date-time feature column
df['timestamp_column'] = pd.to_datetime(df['trans_date_trans_time'])

# Extract date-time features
df['year'] = df['timestamp_column'].dt.year
df['month'] = df['timestamp_column'].dt.month
df['day'] = df['timestamp_column'].dt.day
df['hour'] = df['timestamp_column'].dt.hour
df['minute'] = df['timestamp_column'].dt.minute
df['second'] = df['timestamp_column'].dt.second
df['day_of_week'] = df['timestamp_column'].dt.dayofweek
df['is_weekend'] = df['day_of_week'].apply(lambda x: 1 if x > 4 else 0)

from datetime import datetime

# Assuming 'df' is your DataFrame and 'dob' is the date of birth column
df['dob'] = pd.to_datetime(df['dob'])

# Calculate age in years as of today's date
df['age'] = (datetime.now() - df['dob']).dt.days // 365

# You can now drop the original 'dob' column if you wish
df.drop(columns=['dob'], inplace=True)


# Drop the original date-time column if you no longer need it
df.drop(columns=['trans_date_trans_time'], inplace=True)

# Now df is ready for model training with CatBoost


In [29]:
df.drop(columns=['unix_time'])
df.columns

Index(['Unnamed: 0', 'cc_num', 'merchant', 'category', 'amt', 'first', 'last',
       'gender', 'street', 'city', 'state', 'zip', 'lat', 'long', 'city_pop',
       'job', 'trans_num', 'unix_time', 'merch_lat', 'merch_long', 'is_fraud',
       'timestamp_column', 'year', 'month', 'day', 'hour', 'minute', 'second',
       'day_of_week', 'is_weekend', 'age'],
      dtype='object')

In [30]:
# show proportion of fraud vs non-fraud
df['is_fraud'].value_counts(normalize=True)

is_fraud
0    0.994211
1    0.005789
Name: proportion, dtype: float64

In [31]:
# train test split (stratified based on is_fraud)

from sklearn.model_selection import train_test_split

X = df.drop('is_fraud', axis=1)

y = df['is_fraud']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

# check proportion of fraud in train and test
y_train.value_counts(normalize=True)
y_test.value_counts(normalize=True)


is_fraud
0    0.99421
1    0.00579
Name: proportion, dtype: float64

In [32]:
# initial CatBoost model
from catboost import CatBoostClassifier

# List of categorical columns
categorical_features = [
    'merchant', 'category', 'first', 'last', 'gender',
    'street', 'city', 'state', 'zip', 'job', 'cc_num', 'trans_num'
]

# Initialize CatBoost with the categorical features
catboost_model = CatBoostClassifier(
    auto_class_weights='Balanced',
    cat_features=categorical_features,  # Specify the categorical columns by name
    verbose=0,
    random_state=42
)

# Fit the model with your training data
catboost_model.fit(X_train, y_train)



In [33]:
# check accuracy

from sklearn.metrics import accuracy_score

y_pred = catboost_model.predict(X_test)

accuracy_score(y_test, y_pred)

0.9977604274313707